# Synthetic Regression Data
:label:`sec_synthetic-regression-data`


Machine learning is all about extracting information from data.
So you might wonder, what could we possibly learn from synthetic data?
While we might not care intrinsically about the patterns 
that we ourselves baked into an artificial data generating model,
such datasets are nevertheless useful for didactic purposes,
helping us to evaluate the properties of our learning 
algorithms and to confirm that our implementations work as expected.
For example, if we create data for which the correct parameters are known *a priori*,
then we can check that our model can in fact recover them.

# 합성 회귀 데이터
:label:`sec_synthetic-regression-data`

머신 러닝은 데이터에서 정보를 추출하는 것입니다.그러면 합성 데이터에서 무엇을 배울 수 있을까요?우리는 본질적으로 인공 데이터 생성 모델에 내장된 패턴에 관심이 없을 수도 있지만,
그러한 데이터 세트는 교육적 목적으로는 유용합니다.
학습 알고리즘의 속성을 평가하고 구현이 예상대로 작동하는지 확인하는 데 도움이 됩니다.예를 들어, *사전적으로* 정확한 매개변수가 알려진 데이터를 생성하면
모델이 실제로 해당 매개변수를 복구할 수 있는지 확인할 수 있습니다.


In [1]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

## Generating the Dataset

For this example, we will work in low dimension
for succinctness.
The following code snippet generates 1000 examples
with 2-dimensional features drawn 
from a standard normal distribution.
The resulting design matrix $\mathbf{X}$
belongs to $\mathbb{R}^{1000 \times 2}$. 
We generate each label by applying 
a *ground truth* linear function, 
corrupting them via additive noise $\boldsymbol{\epsilon}$, 
drawn independently and identically for each example:

(**$$\mathbf{y}= \mathbf{X} \mathbf{w} + b + \boldsymbol{\epsilon}.$$**)

For convenience we assume that $\boldsymbol{\epsilon}$ is drawn 
from a normal distribution with mean $\mu= 0$ 
and standard deviation $\sigma = 0.01$.
Note that for object-oriented design
we add the code to the `__init__` method of a subclass of `d2l.DataModule` (introduced in :numref:`oo-design-data`). 
It is good practice to allow the setting of any additional hyperparameters. 
We accomplish this with `save_hyperparameters()`. 
The `batch_size` will be determined later.

## 데이터셋 생성

이 예제에서는 간결성을 위해 저차원에서 작업하겠습니다.다음 코드 조각은 표준 정규 분포에서 추출된 2차원 특성을 가진 1000개의 예제를 생성합니다.결과적인 설계 행렬 $\mathbf{X}$는 $\mathbb{R}^{1000 \times 2}$에 속합니다.각 레이블은
*기준 진실* 선형 함수를 적용하고,
가산 잡음 $\boldsymbol{\epsilon}$을 통해 손상시켜 생성합니다.
각 예제에 대해 독립적이고 동일하게 추출됩니다.

(**$$\mathbf{y}= \mathbf{X} \mathbf{w} + b + \boldsymbol{\epsilon}.$$**)

편의상 $\boldsymbol{\epsilon}$은 평균 $\mu= 0$이고 표준 편차 $\sigma = 0.01$인 정규 분포에서 추출되었다고 가정합니다.객체 지향 설계의 경우,
`d2l.DataModule`의 하위 클래스인 `__init__` 메서드(:numref:`oo-design-data`에서 도입됨)에 코드를 추가합니다.추가 하이퍼파라미터 설정을 허용하는 것이 좋습니다.`save_hyperparameters()`를 사용하여 이를 수행합니다.`batch_size`는 나중에 결정됩니다.


In [2]:
class SyntheticRegressionData(d2l.DataModule):  #@save
    """Synthetic data for linear regression."""
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=1000,
                 batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise

Below, we set the true parameters to $\mathbf{w} = [2, -3.4]^\top$ and $b = 4.2$.
Later, we can check our estimated parameters against these *ground truth* values.

아래에서 실제 매개변수를 $\mathbf{w} = [2, -3.4]^\top$ 및 $b = 4.2$로 설정합니다.나중에 추정된 매개변수를 이러한 *실제* 값과 비교할 수 있습니다.


In [3]:
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)

[**Each row in `features` consists of a vector in $\mathbb{R}^2$ and each row in `labels` is a scalar.**] Let's have a look at the first entry.

`features`의 각 행은 $\mathbb{R}^2$의 벡터로 구성되고 `labels`의 각 행은 스칼라입니다. 첫 번째 항목을 살펴보겠습니다.


In [4]:
print('features:', data.X[0],'\nlabel:', data.y[0])

features: tensor([ 0.1966, -0.6143]) 
label: tensor([6.6860])


## Reading the Dataset

Training machine learning models often requires multiple passes over a dataset, 
grabbing one minibatch of examples at a time. 
This data is then used to update the model. 
To illustrate how this works, we 
[**implement the `get_dataloader` method,**] 
registering it in the `SyntheticRegressionData` class via `add_to_class` (introduced in :numref:`oo-design-utilities`).
It (**takes a batch size, a matrix of features,
and a vector of labels, and generates minibatches of size `batch_size`.**)
As such, each minibatch consists of a tuple of features and labels. 
Note that we need to be mindful of whether we're in training or validation mode: 
in the former, we will want to read the data in random order, 
whereas for the latter, being able to read data in a pre-defined order 
may be important for debugging purposes.

## 데이터셋 읽기

머신 러닝 모델을 학습하려면 데이터셋을 여러 번 반복해야 하며,
한 번에 하나의 미니 배치(minibatch) 예제를 가져옵니다.이 데이터는 모델을 업데이트하는 데 사용됩니다.이 동작 방식을 설명하기 위해
[**`get_dataloader` 메서드를 구현하고,**]
`add_to_class`(:numref:`oo-design-utilities`에서 소개됨)를 통해 `SyntheticRegressionData` 클래스에 등록합니다.(**배치 크기, 특성 행렬, 그리고 레이블 벡터를 받아서 `batch_size` 크기의 미니 배치를 생성합니다.**)
따라서 각 미니 배치는 특성과 레이블의 튜플로 구성됩니다.학습 모드인지 검증 모드인지 주의해야 합니다. 학습 모드에서는 데이터를 무작위 순서로 읽어야 하지만, 검증 모드에서는 미리 정의된 순서로 데이터를 읽는 것이 디버깅에 중요할 수 있습니다.


In [5]:
@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train):
    if train:
        indices = list(range(0, self.num_train))
        # The examples are read in random order
        random.shuffle(indices)
    else:
        indices = list(range(self.num_train, self.num_train+self.num_val))
    for i in range(0, len(indices), self.batch_size):
        batch_indices = torch.tensor(indices[i: i+self.batch_size])
        yield self.X[batch_indices], self.y[batch_indices]

To build some intuition, let's inspect the first minibatch of
data. Each minibatch of features provides us with both its size and the dimensionality of input features.
Likewise, our minibatch of labels will have a matching shape given by `batch_size`.

직관을 키우기 위해 첫 번째 미니배치를 살펴보겠습니다.데이터의 각 미니배치는 입력 특성의 크기와 차원을 모두 제공합니다.마찬가지로, 레이블의 미니배치는 `batch_size`에 따라 일치하는 모양을 갖습니다.


In [6]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


While seemingly innocuous, the invocation 
of `iter(data.train_dataloader())` 
illustrates the power of Python's object-oriented design. 
Note that we added a method to the `SyntheticRegressionData` class
*after* creating the `data` object. 
Nonetheless, the object benefits from 
the *ex post facto* addition of functionality to the class.

Throughout the iteration we obtain distinct minibatches
until the entire dataset has been exhausted (try this).
While the iteration implemented above is good for didactic purposes,
it is inefficient in ways that might get us into trouble with real problems.
For example, it requires that we load all the data in memory
and that we perform lots of random memory access.
The built-in iterators implemented in a deep learning framework
are considerably more efficient and they can deal
with sources such as data stored in files, 
data received via a stream, 
and data generated or processed on the fly. 
Next let's try to implement the same method using built-in iterators.

## Concise Implementation of the Data Loader

Rather than writing our own iterator,
we can [**call the existing API in a framework to load data.**]
As before, we need a dataset with features `X` and labels `y`. 
Beyond that, we set `batch_size` in the built-in data loader 
and let it take care of shuffling examples  efficiently.

겉보기에는 무해해 보이지만, `iter(data.train_dataloader())` 호출은 Python 객체 지향 설계의 강력함을 보여줍니다.`data` 객체를 생성한 *후에* `SyntheticRegressionData` 클래스에 메서드를 추가했습니다.그럼에도 불구하고, 이 객체는 클래스에 *사후적으로* 기능을 추가함으로써 이점을 얻습니다.반복 과정 전체에서 전체 데이터셋이 소진될 때까지 개별 미니배치를 얻습니다(다음 방법을 시도해 보세요).위에서 구현한 반복은 학습용으로는 좋지만,
실제 문제에 직면하게 될 수 있는 방식으로 비효율적입니다.예를 들어, 모든 데이터를 메모리에 로드하고
수많은 임의 메모리 접근을 수행해야 합니다.딥러닝 프레임워크에 구현된 내장 반복자는 훨씬 더 효율적이며
파일에 저장된 데이터,
스트림을 통해 수신된 데이터,
즉시 생성되거나 처리된 데이터와 같은 소스를 처리할 수 있습니다.다음으로, 내장 반복자를 사용하여 동일한 메서드를 구현해 보겠습니다.## 데이터 로더의 간결한 구현

직접 반복자를 작성하는 대신
[**프레임워크에서 기존 API를 호출하여 데이터를 로드할 수 있습니다.**]
이전과 마찬가지로, 피처 `X`와 레이블 `y`가 있는 데이터 세트가 필요합니다.그 외에도, 내장 데이터 로더에 `batch_size`를 설정하고
예제 셔플링을 효율적으로 처리하도록 합니다.


In [7]:
@d2l.add_to_class(d2l.DataModule)  #@save
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    dataset = torch.utils.data.TensorDataset(*tensors)
    return torch.utils.data.DataLoader(dataset, self.batch_size,
                                       shuffle=train)

In [8]:
@d2l.add_to_class(SyntheticRegressionData)  #@save
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)

The new data loader behaves just like the previous one, except that it is more efficient and has some added functionality.

새로운 데이터 로더는 이전 데이터 로더와 동일하게 동작하지만, 효율성이 더 높고 몇 가지 추가 기능이 있습니다.


In [9]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


For instance, the data loader provided by the framework API 
supports the built-in `__len__` method, 
so we can query its length, 
i.e., the number of batches.

예를 들어, 프레임워크 API가 제공하는 데이터 로더는 내장된 `__len__` 메서드를 지원하므로,
길이, 즉 배치 수를 쿼리할 수 있습니다.


In [10]:
len(data.train_dataloader())

32

## Summary

Data loaders are a convenient way of abstracting out 
the process of loading and manipulating data. 
This way the same machine learning *algorithm* 
is capable of processing many different types and sources of data 
without the need for modification. 
One of the nice things about data loaders 
is that they can be composed. 
For instance, we might be loading images 
and then have a postprocessing filter 
that crops them or modifies them in other ways. 
As such, data loaders can be used 
to describe an entire data processing pipeline. 

As for the model itself, the two-dimensional linear model 
is about the simplest we might encounter. 
It lets us test out the accuracy of regression models 
without worrying about having insufficient amounts of data 
or an underdetermined system of equations. 
We will put this to good use in the next section.  


## Exercises

1. What will happen if the number of examples cannot be divided by the batch size. How would you change this behavior by specifying a different argument by using the framework's API?
1. Suppose that we want to generate a huge dataset, where both the size of the parameter vector `w` and the number of examples `num_examples` are large.
    1. What happens if we cannot hold all data in memory?
    1. How would you shuffle the data if it is held on disk? Your task is to design an *efficient* algorithm that does not require too many random reads or writes. Hint: [pseudorandom permutation generators](https://en.wikipedia.org/wiki/Pseudorandom_permutation) allow you to design a reshuffle without the need to store the permutation table explicitly :cite:`Naor.Reingold.1999`. 
1. Implement a data generator that produces new data on the fly, every time the iterator is called. 
1. How would you design a random data generator that generates *the same* data each time it is called?

## 요약

데이터 로더는 데이터 로딩 및 조작 프로세스를 추상화하는 편리한 방법입니다.이렇게 하면 동일한 머신 러닝 *알고리즘*으로 다양한 유형과 소스의 데이터를 수정 없이 처리할 수 있습니다.데이터 로더의 장점 중 하나는 구성이 가능하다는 것입니다.예를 들어, 이미지를 로딩한 후
이미지를 자르거나 다른 방식으로 수정하는 후처리 필터를 적용할 수 있습니다.따라서 데이터 로더를 사용하여 전체 데이터 처리 파이프라인을 설명할 수 있습니다.모델 자체에 관해서는 2차원 선형 모델이
가장 간단한 모델입니다.데이터 양이 부족하거나 방정식 시스템이 불확실하다는 걱정 없이
회귀 모델의 정확도를 테스트할 수 있습니다.다음 섹션에서 이 기능을 유용하게 활용하겠습니다.## 연습

1. 예제 개수를 배치 크기로 나눌 수 없는 경우 어떻게 될까요?프레임워크의 API를 사용하여 다른 인수를 지정하여 이러한 동작을 어떻게 변경할 수 있을까요?1. 매개변수 벡터 `w`의 크기와 예제 개수 `num_examples`가 모두 큰 거대한 데이터 세트를 생성한다고 가정해 보겠습니다.1. 모든 데이터를 메모리에 저장할 수 없는 경우 어떻게 될까요?1. 데이터가 디스크에 저장되어 있는 경우 어떻게 섞을까요?무작위 읽기나 쓰기를 너무 많이 요구하지 않는 *효율적인* 알고리즘을 설계하는 것이 과제입니다.힌트: [의사 난수 생성기](https://en.wikipedia.org/wiki/Pseudorandom_permutation)를 사용하면 순열 테이블을 명시적으로 저장할 필요 없이 재혼합을 설계할 수 있습니다.(:cite:`Naor.Reingold.1999`)
1. 반복자가 호출될 때마다 새로운 데이터를 즉시 생성하는 데이터 생성기를 구현하세요.1. 호출될 때마다 *동일한* 데이터를 생성하는 무작위 데이터 생성기를 설계하려면 어떻게 해야 할까요?


데이터 로딩과 관련하여 여러 질문을 하셨네요. 각 질문에 대해 자세히 답변해 드리겠습니다.

-----

## 1\. 예제 수가 배치 크기로 나누어 떨어지지 않는 경우 📊

**무슨 일이 일어나는가?**
일반적으로 데이터셋의 예제 수(`num_examples`)가 배치 크기(`batch_size`)로 나누어 떨어지지 않을 때, 마지막 배치는 `batch_size`보다 **작은 수의 예제**를 포함하게 됩니다. 예를 들어, 예제가 100개이고 배치 크기가 16이라면, 첫 6개 배치는 각각 16개의 예제를 가지고, 마지막 7번째 배치는 4개의 예제($100 - 6 \\times 16 = 4$)를 가지게 됩니다.

**프레임워크 API를 사용하여 이 동작을 변경하는 방법:**
대부분의 딥러닝 프레임워크(PyTorch의 `DataLoader`, TensorFlow의 `tf.data.Dataset` 등)는 이러한 경우를 처리하기 위한 인자를 제공합니다. 일반적으로 `drop_last` 또는 이와 유사한 이름의 인자를 사용합니다.

  * **`drop_last=True` (또는 해당 프레임워크의 동등한 인자):** 이 인자를 `True`로 설정하면, 마지막에 배치 크기보다 작은 수의 예제를 포함하는 불완전한 배치는 **버려집니다**. 이 경우 모든 배치는 정확히 `batch_size`만큼의 예제를 가지게 됩니다. 이는 학습 과정에서 배치 크기를 일정하게 유지하여 배치 정규화(Batch Normalization) 등의 특정 연산에서 일관성을 유지하는 데 유용할 수 있습니다.

예시 (PyTorch):

```python
import torch
from torch.utils.data import DataLoader, TensorDataset

# 예제 데이터: 100개의 데이터, 1개의 특성
data = torch.randn(100, 1)
labels = torch.randint(0, 2, (100, 1))
dataset = TensorDataset(data, labels)

batch_size = 16

# drop_last=False (기본값): 마지막 배치에 4개의 예제 포함
dataloader_default = DataLoader(dataset, batch_size=batch_size, shuffle=False)
print(f"Default DataLoader (drop_last=False):")
for i, batch in enumerate(dataloader_default):
    print(f"Batch {i+1} size: {len(batch[0])}")
# 출력: Batch 1 size: 16, ..., Batch 6 size: 16, Batch 7 size: 4

print("-" * 30)

# drop_last=True: 마지막 불완전한 배치 버림
dataloader_drop_last = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=True)
print(f"DataLoader with drop_last=True:")
for i, batch in enumerate(dataloader_drop_last):
    print(f"Batch {i+1} size: {len(batch[0])}")
# 출력: Batch 1 size: 16, ..., Batch 6 size: 16 (7번째 배치는 버려짐)
```

-----

## 2\. 모든 데이터를 메모리에 담을 수 없는 경우 💾

매개변수 벡터 $w$의 크기와 예제 수(`num_examples`)가 모두 커서 모든 데이터를 메모리에 한꺼번에 로드할 수 없는 상황은 대규모 딥러닝 프로젝트에서 흔히 발생합니다.

**이 경우 발생하는 문제:**
가장 큰 문제는 \*\*`MemoryError` (메모리 부족 오류)\*\*가 발생하여 프로그램을 실행할 수 없다는 것입니다. 이는 데이터 로딩 단계에서 전체 데이터셋을 한 번에 RAM으로 읽어들이려고 시도할 때 발생합니다. 설령 오류가 발생하지 않더라도, 시스템이 스왑 메모리(디스크 공간)를 사용하게 되어 데이터 접근 속도가 극도로 느려지므로 사실상 학습이 불가능해집니다.

**해결책:**
이러한 문제를 해결하기 위해 **디스크 기반 데이터 로딩 및 스트리밍** 기술을 사용해야 합니다.

  * **데이터 제너레이터(Data Generator) 또는이터레이터(Iterator):** 필요한 데이터를 메모리로 한 번에 로드하는 대신, 모델 학습에 필요한 작은 데이터 청크(예: 미니 배치)만 디스크에서 읽어와 메모리에 로드합니다. 각 학습 스텝마다 새로운 미니 배치를 요청하면 제너레이터가 이를 제공합니다.
  * **파일 형식 최적화:** 데이터를 효율적으로 읽을 수 있는 형식(예: TFRecord, HDF5, Parquet)으로 저장합니다. 이러한 형식은 대규모 데이터셋을 위한 인덱싱 및 압축 기능을 제공하여 I/O 성능을 향상시킵니다.
  * **메모리 매핑(Memory Mapping):** 파일의 일부 또는 전체를 가상 메모리 주소 공간에 매핑하여 마치 메모리에 있는 것처럼 직접 접근할 수 있도록 합니다. 이는 큰 파일의 일부만 읽어올 때 유용합니다.
  * **분산 처리:** 여러 머신에 데이터를 분산 저장하고, 각 머신이 자체 데이터 청크를 처리하도록 하여 전체 데이터 로딩 및 처리 부담을 분산시킵니다.

-----

## 3\. 디스크에 저장된 데이터를 효율적으로 셔플링하는 방법 🔀

디스크에 저장된 대규모 데이터를 효율적으로 셔플링하려면, 모든 데이터를 메모리에 로드하지 않고도 무작위 순서로 접근할 수 있는 알고리즘이 필요합니다. "Pseudorandom permutation generators" 힌트를 바탕으로, 다음과 같은 접근 방식을 설계할 수 있습니다.

**알고리즘 설계: 의사 난수 순열 생성기 기반 셔플링**

1.  **데이터 파일 구조:**

      * 모든 데이터 예제는 고유한 **인덱스**를 가집니다 (예: 0부터 `num_examples - 1`까지).
      * 각 예제는 디스크의 특정 위치에 저장됩니다. (예: 각 예제가 하나의 파일이거나, 큰 파일 내의 고정된 오프셋에 저장)

2.  **의사 난수 순열 생성기:**

      * `P(i)`는 입력 인덱스 `i`를 받아 `0`부터 `num_examples - 1`까지의 범위 내에서 다른 고유한 인덱스를 출력하는 함수라고 합시다. 이 함수는 순열(permutation)을 생성하므로, 모든 `i`에 대해 `P(i)`는 서로 다른 값을 반환하고 모든 가능한 출력 인덱스를 한 번씩만 생성합니다.
      * **핵심 아이디어:** `P(i)`는 **명시적인 순열 테이블(permutation table)을 저장하지 않고도** `i`로부터 결정론적으로 `P(i)`를 계산할 수 있어야 합니다. 이는 특정 암호화 알고리즘(예: Feistel network 기반의 순열 또는 블록 암호의 변형)을 사용하여 구현할 수 있습니다.
      * **예시 (간단한 형태의 유사 순열 함수):**
          * `P(i) = (a * i + b) % num_examples` (이는 완벽한 순열이 아닐 수 있으며, `num_examples`가 소수이고 `a`가 `num_examples`와 서로소일 때만 작동함).
          * 더 견고한 방법은 특정 길이의 블록 암호(예: AES-128)를 사용하여 인덱스를 암호화하는 것입니다. 인덱스 `i`를 암호화 키로 사용하거나, `i`를 블록으로 사용하여 고정된 키로 암호화한 후 결과를 `num_examples`에 맞게 매핑하는 방식입니다. 여기서 핵심은 \*\*단방향이 아닌 양방향 매핑(역함수가 존재)\*\*이고 \*\*전단사(bijective)\*\*여야 한다는 점입니다.

3.  **셔플링 알고리즘:**
    a.  **초기화:** 난수 시드(seed)와 필요한 `P(i)` 함수에 사용될 매개변수를 설정합니다.
    b.  **반복:** 각 에포크(epoch)에서:
    i.  `0`부터 `num_examples - 1`까지의 정수 시퀀스를 생성합니다.
    ii. 이 시퀀스의 각 인덱스 `i`에 대해 `P(i)`를 계산하여 **셔플된 논리적 인덱스** `shuffled_idx = P(i)`를 얻습니다.
    iii. `shuffled_idx`에 해당하는 데이터를 디스크에서 읽어 메모리로 로드합니다.
    iv. 이 로드된 데이터들을 **배치 크기만큼 그룹화**하여 모델에 제공합니다.

**효율성 분석:**

  * **적은 무작위 읽기/쓰기:** 이 방법은 전체 데이터셋에 대한 무작위 읽기/쓰기를 요구하지 않습니다. 특정 순서에 따라 디스크에서 데이터 블록을 읽어올 수 있습니다 (예: 데이터를 인덱스 순서로 저장한 후, `P(i)`가 생성하는 인덱스에 따라 데이터를 찾아 읽기).
  * **메모리 효율성:** 순열 테이블을 명시적으로 저장할 필요가 없으므로, 메모리 사용량이 극적으로 줄어듭니다. `num_examples`가 아무리 커도 `P(i)` 함수와 현재 인덱스만 메모리에 있으면 됩니다.
  * **재현 가능성:** 난수 시드를 고정하면 `P(i)` 함수의 출력이 매번 동일하므로, 데이터 셔플링 순서를 재현할 수 있습니다.

-----

## 4\. 즉석에서 새 데이터를 생성하는 데이터 제너레이터 구현 🧪

여기서는 Python의 제너레이터(generator)를 사용하여 `__iter__` 메서드가 호출될 때마다 새로운 데이터를 생성하는 `DataGenerator` 클래스를 구현합니다. 이는 특히 시뮬레이션 데이터나 강화 학습 환경에서 유용합니다.

```python
import random
import numpy as np

class RealtimeDataGenerator:
    def __init__(self, num_features, data_range=(0, 1), label_range=(0, 1), seed=None):
        self.num_features = num_features
        self.data_range = data_range
        self.label_range = label_range
        self.seed = seed
        self.rng = random.Random(self.seed) # 각 인스턴스마다 별도의 RNG

    def __iter__(self):
        """
        이터레이터가 호출될 때마다 새로운 데이터를 무한히 생성합니다.
        yield를 사용하여 각 호출에서 하나의 (데이터, 레이블) 쌍을 반환합니다.
        """
        while True:
            # num_features 만큼 무작위 데이터 생성
            data = [self.rng.uniform(self.data_range[0], self.data_range[1]) for _ in range(self.num_features)]
            # 무작위 레이블 생성
            label = self.rng.uniform(self.label_range[0], self.label_range[1])
            yield np.array(data, dtype=np.float32), np.array(label, dtype=np.float32)

# 사용 예시:
print("--- 실시간 데이터 제너레이터 ---")
generator_1 = RealtimeDataGenerator(num_features=5, seed=42)
iterator_1 = iter(generator_1)

# 첫 3개의 데이터 포인트 출력
for _ in range(3):
    data, label = next(iterator_1)
    print(f"Data: {data}, Label: {label}")

print("-" * 20)

# 같은 제너레이터 객체에서 다시 이터레이터를 얻으면 새로운 시퀀스가 시작됨 (아님, 계속 이어짐)
# D2L이나 PyTorch DataLoader처럼 동작하게 하려면 __next__ 구현 필요
# 또는 아래와 같이 새 이터레이터 객체 생성
iterator_2 = iter(generator_1) # 같은 generator_1 객체에서 새 이터레이터 생성
print("--- 같은 generator_1 객체에서 다시 얻은 이터레이터 (계속 이어서 생성) ---")
for _ in range(3):
    data, label = next(iterator_2)
    print(f"Data: {data}, Label: {label}")

print("-" * 20)
# 참고: 이 구현에서는 __iter__가 무한 루프를 돌기 때문에,
# 매번 새로운 이터레이터 객체를 얻으면 계속 이어서 데이터를 생성합니다.
# 일반적으로 Dataloader는 데이터를 다 소진하면 StopIteration을 발생시킵니다.
# 이는 '무한히 새로운 데이터를 생성'하라는 요구사항에 부합합니다.
```

-----

## 5\. 호출될 때마다 동일한 데이터를 생성하는 무작위 데이터 제너레이터 설계 🔄

동일한 무작위 데이터를 재현 가능하게 생성하려면, 난수 생성기(Random Number Generator, RNG)에 \*\*고정된 시드(seed)\*\*를 제공해야 합니다. 이렇게 하면 RNG가 항상 동일한 난수 시퀀스를 생성하므로, 결과적으로 동일한 데이터를 생성합니다.

```python
import numpy as np

class ReproducibleDataGenerator:
    def __init__(self, num_examples, num_features, seed=42):
        self.num_examples = num_examples
        self.num_features = num_features
        self.seed = seed
        self._current_idx = 0 # 현재 생성된 예제의 인덱스

    def __iter__(self):
        """
        이터레이터를 초기화할 때마다 난수 생성기를 재설정하여
        매번 동일한 데이터 시퀀스를 생성하도록 합니다.
        """
        # __iter__가 호출될 때마다 난수 생성기를 고정된 시드로 초기화
        self.rng_state = np.random.get_state() # 현재 numpy RNG 상태 저장
        np.random.seed(self.seed) # 고정된 시드로 numpy RNG 초기화
        self._current_idx = 0 # 예제 인덱스 리셋
        return self

    def __next__(self):
        """
        요청 시 데이터를 생성하고, num_examples만큼 생성되면 StopIteration을 발생시킵니다.
        """
        if self._current_idx < self.num_examples:
            # 고정된 시드로 생성되므로 매번 동일한 무작위 데이터가 나옴
            data = np.random.rand(self.num_features).astype(np.float32)
            label = np.random.rand(1).astype(np.float32) # 레이블도 무작위로 생성

            self._current_idx += 1
            return data, label
        else:
            # 모든 예제를 생성했으면 원래 numpy RNG 상태로 복원 (선택 사항이지만 좋은 관행)
            np.random.set_state(self.rng_state)
            raise StopIteration

# 사용 예시:
print("--- 재현 가능한 데이터 제너레이터 ---")
gen_reproducible = ReproducibleDataGenerator(num_examples=5, num_features=3, seed=123)

print("첫 번째 이터레이션:")
for i, (data, label) in enumerate(gen_reproducible):
    print(f"Example {i+1}: Data: {data}, Label: {label}")

print("-" * 20)

print("두 번째 이터레이션 (동일한 결과 기대):")
for i, (data, label) in enumerate(gen_reproducible):
    print(f"Example {i+1}: Data: {data}, Label: {label}")

print("-" * 20)

print("새로운 인스턴스로 다른 시드 사용:")
gen_another_seed = ReproducibleDataGenerator(num_examples=3, num_features=2, seed=456)
for i, (data, label) in enumerate(gen_another_seed):
    print(f"Example {i+1}: Data: {data}, Label: {label}")

```

**핵심:** `__iter__` 메서드 내에서 `np.random.seed(self.seed)`를 호출하여, 이터레이터가 새로 생성될 때마다 난수 생성기의 상태를 고정된 시드로 재설정합니다. 이렇게 하면 `__next__`를 통해 생성되는 "무작위" 데이터 시퀀스가 매번 동일하게 유지됩니다. 🚀

[Discussions](https://discuss.d2l.ai/t/6663)

[토론](https://discuss.d2l.ai/t/6663)
